In [ ]:

import pandas as pd
import json
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm
import re

# --- Data Preparation ---

# load a different sample for the main run of my script.
df = pd.read_csv("/Users/shangray/Desktop/amazon_sample_10000.csv")

# Grab the 'review_text' column and convert it to a list.
input_reviews = df["review_text"].tolist()
# For a quick test, I'll just work with the first few.
# input_reviews = input_reviews[0:4]

# Just taking a quick look at the first few items to make sure it loaded correctly.
print(input_reviews[:5])

# --- Model & Function Setup ---
# Setting up the T5 models I need. I'm using two stages here.
# Stage 1: This model classifies sentences from a review.
# Stage 2: This one summarizes the results from Stage 1.

# I'll define the paths to my trained models.
stage1_path = "/Users/shangray/Desktop/sch_project/t5_stage1_trained"
stage2_path = "/Users/shangray/Desktop/sch_project/t5_stage2_trained_local"

In [ ]:
#loading the tokenizers and models for each stage.
#use MPS if it's available for faster processing on my Mac, otherwise I'll use the CPU.
device = "mps" if torch.backends.mps.is_available() else "cpu"

tokenizer1 = T5Tokenizer.from_pretrained(stage1_path)
model1 = T5ForConditionalGeneration.from_pretrained(stage1_path).to(device)

tokenizer2 = T5Tokenizer.from_pretrained(stage2_path)
model2 = T5ForConditionalGeneration.from_pretrained(stage2_path).to(device)


def predict(text, model, tokenizer, max_length=512):
    """
    I wrote this simple helper function to get predictions from a T5 model.
    It takes text, tokenizes it, and returns the generated output.
    """
    inputs = tokenizer(text, return_tensors="pt").to(device)
    outputs = model.generate(inputs["input_ids"], max_length=max_length)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


def parse_output(output_str):
    """
    I created this function to parse the model's output string into a dictionary.
    The format is expected to be 'positive: ...; negative: ...'
    """
    # I'll use a regex to capture the text for the 'positive' and 'negative' sections.
    match = re.search(r"positive:\s*(.*?)\s*negative:\s*(.*)", output_str, re.IGNORECASE | re.DOTALL)
    if not match:
        return {"positive": [], "negative": []}

    pos_text, neg_text = match.groups()

    # I'll split the text by semicolons and clean up any extra whitespace.
    positives = [p.strip() for p in pos_text.split(";") if p.strip()]
    negatives = [n.strip() for n in neg_text.split(";") if n.strip()]

    return {"positive": positives, "negative": negatives}


# --- Setting up the Evaluation Tool (DistilBERT) ---
# Using a pre-trained sentiment analysis model (DistilBERT) to evaluate

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
bert_tokenizer = AutoTokenizer.from_pretrained(model_name)
bert_model = AutoModelForSequenceClassification.from_pretrained(model_name)
bert_model.to(device)


def classify_sentiment(sentence: str) -> str:
    """
    I built this function to classify a sentence as 'positive' or 'negative' using DistilBERT.
    """
    inputs = bert_tokenizer(sentence, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    prediction_id = torch.argmax(outputs.logits, dim=-1).item()
    return ["negative", "positive"][prediction_id]


# --- Main Inference & Evaluation Loop ---
# Run my reviews through the two T5 models and see what the results are.
# Use DistilBERT to calculate an accuracy score.

total_sentences = 0
correct_predictions = 0

for idx, review in enumerate(tqdm(input_reviews, desc="Predicting")):
    print(f"\n--- Processing Review {idx + 1} ---")
    print(f"Input:\n{review}")

    # Stage 1: get the sentence classifications (pros and cons).
    stage1_output = predict(review, model1, tokenizer1)
    
    # parse the output into a dictionary to make it easy to work with.
    parsed = parse_output(stage1_output)
    
    print(f"\nStage 1 Output:\n{stage1_output}")
    
    # check the accuracy of the Stage 1 output against DistilBERT.
    # iterate through the classified sentences and compare them.
    for label, sentences in parsed.items():
        for sentence in sentences:
            total_sentences += 1
            bert_label = classify_sentiment(sentence)
            if bert_label == label:
                correct_predictions += 1
            # You can uncomment the line below for a more detailed look at the comparison.
            # print(f"{sentence} | T5: {label} | BERT: {bert_label}")

    # Stage 2: summarize the pros and cons that were identified in Stage 1.
    stage2_output = predict(stage1_output, model2, tokenizer2)
    print(f"\nStage 2 Output:\n{stage2_output}")

# --- Final Results ---
# calculate and print the final accuracy score.
accuracy = correct_predictions / total_sentences if total_sentences > 0 else 0
print(f"\nAccuracy of Stage 1 model against DistilBERT = {accuracy:.2%}")

# save the accuracy score to a file.
file_name = "Stage1_accuracy.txt"
with open(file_name, 'w') as file:
    # I need to convert the float to a string before I can write it to the file.
    file.write(str(accuracy))

print(f"Accuracy value successfully saved to '{file_name}'.")

/Users/shangray/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/shangray/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


["Hooked from the beginning, page one, I could not put this book down. I identified with every thought, word, and feeling. I was amazed by the courage of the writer, and his strength in sticking to his gut feelings in a recovery world that tells one that one's own best thinking landed them in the place that they occupy! That he did this with courage, dignity, fear,guilt, great love and sick will gave me a feeling of great hope for him, and all human beings at a particularly negative time in human history! The depth and breadth of his honesty was a homecoming for me and many like me.", 'This one gift I could give my grown daughter. I read this book over and over to her as a child. It was in wonderful condition!', "It's hard to appreciate Danzig anymore since his voice is shot, this cd proved it, I miss the old days.................", 'I heard about this book and thought, "Wow. Should be great. Here\'s a guy who endured a major tragedy (losing both parents within a month). He should have

Predicting:   0%|          | 0/10000 [00:00<?, ?it/s]


--- Processing Review 1 ---
Input:
Hooked from the beginning, page one, I could not put this book down. I identified with every thought, word, and feeling. I was amazed by the courage of the writer, and his strength in sticking to his gut feelings in a recovery world that tells one that one's own best thinking landed them in the place that they occupy! That he did this with courage, dignity, fear,guilt, great love and sick will gave me a feeling of great hope for him, and all human beings at a particularly negative time in human history! The depth and breadth of his honesty was a homecoming for me and many like me.

Stage 1 Output:
positive: Identified with every thought, word, and feeling; Courage and strength of the writer; Strength in sticking to gut feelings in a recovery world; Courage, dignity, fear, guilt, great love, and sick will; Honesty and depth of honesty; Homecoming for many negative experiences negative: None


Predicting:   0%|          | 1/10000 [00:06<16:56:17,  6.10s/it]

...


Stage 2 Output:
: None negative: lack of truth-oriented behavior, manipulative quotes and facts, misleading statements about the right, unsubstantiated assertions, lack of evidence, support for mass murder, criticism of Vietnam vets for "treasonous remarks"

--- Processing Review 19 ---
Input:
This is one of the most [boring book] I ever tried to read. I couldn't take any more than the first 25 pages. I would like to set it on fire...that would be about the only thing that would make this nasty little piece of work interesting. I regret spending any money on this book. It's not good enough to give away...they should pay people to take it.

Stage 1 Output:
positive: None negative: The book is very boring and difficult to read.; The first 25 pages are too long for the reviewer.; The reviewer regrets spending money on the book.; The book is not good enough to give away.; The reviewer wants to set the book on fire.


Predicting:   0%|          | 19/10000 [00:56<8:26:41,  3.05s/it]


Stage 2 Output:
Pros: None negative: boring and difficult to read, excessively long first pages, regret over spending, inadequate value for giveaways, desire to set the book on fire.

--- Processing Review 20 ---
Input:
I have been an E. Lynn Harris fan for years! This latest book was just not interesting to me. It seemed like it was the same old plot, nothing really new or exciting. Normally, I read his books within a couple of days. This one was kind of boring to me. Maybe I was expecting too much, but this one didn't do it for me. The characters lacked depth and the plot wasn't juicy like in his last books. I'd hate to stop being a fan and hopefully I'm not "outgrowing" his books. PLEASE produce something better next time, Mr. Harris!

Stage 1 Output:
positive: E. Lynn Harris fans for years; Normally read his books within a couple of days negative: Newer book was boring; Characters lacked depth; Plot wasn't juicy like in previous books; Overall experience was boring


Predicting:   0%|          | 19/10000 [00:58<8:30:26,  3.07s/it]


KeyboardInterrupt: 